<a href="https://colab.research.google.com/github/NasimRidoy/Ingredient-extraction/blob/main/Ingredient_extraction_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# !pip uninstall -y torch torchvision torchaudio
# !pip install torch==2.7.1 torchvision==0.22.1 torchaudio==2.7.1


# !pip install transformers==4.53.3
# !pip install datasets evaluate -q
# !pip install seqeval
# !pip install surya-ocr
!pip install opencv-python numpy

import transformers
print(transformers.__version__) # 4.53.3 for surya-ocr
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)
import torchaudio
print(torchaudio.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.5 MB/s eta 0:00:00
4.55.2
2.6.0+cu124
0.21.0+cu124
2.6.0+cu124


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


detects lines perfect. gives output image and json. BUT TAKES TIME (35 sec in CPU runtime)

In [ ]:
# !surya_detect /content/final_processed_image1.png --images

In [ ]:
import os
import json
from PIL import Image, ImageDraw
from surya.detection import DetectionPredictor

# --- Input image ---
IMAGE_PATH = "/content/drive/MyDrive/Thesis/surya/surya_input_images/in1.jpg"

# --- Output directory ---
detected_dir = os.path.join(os.path.dirname(IMAGE_PATH), "detected_images")
os.makedirs(detected_dir, exist_ok=True)

# --- Initialize detection predictor ---
detection_predictor = DetectionPredictor(device="cuda")

# --- Open image ---
image = Image.open(IMAGE_PATH).convert("RGB")

# --- Run text line detection ---
detections = detection_predictor([image])[0]  # returns TextDetectionResult

# --- Prepare output file paths ---
img_name = os.path.splitext(os.path.basename(IMAGE_PATH))[0]
img_out_path = os.path.join(detected_dir, f"{img_name}_detected.jpg")
json_out_path = os.path.join(detected_dir, f"{img_name}_detections.json")

# --- Draw bounding boxes on image ---
draw = ImageDraw.Draw(image)
for polybox in detections.bboxes:
    x_min, y_min, x_max, y_max = map(int, polybox.bbox)
    draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
    draw.text((x_min, max(0, y_min - 10)), f"{polybox.confidence:.2f}", fill="red")

# --- Save annotated image ---
image.save(img_out_path)

# --- Save detections as JSON ---
detections_list = []
for polybox in detections.bboxes:
    detections_list.append({
        "polygon": polybox.polygon,
        "confidence": float(polybox.confidence),
        "bbox": list(map(int, polybox.bbox))
    })

with open(json_out_path, "w") as f:
    json.dump(detections_list, f, indent=4)

print(f"✅ Processed: {img_name}")
print(f"   Saved detected image → {img_out_path}")
print(f"   Saved detections JSON → {json_out_path}")


Detecting bboxes: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]

✅ Processed: in1
   Saved detected image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1_detected.jpg
   Saved detections JSON → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1_detections.json


for looping all the images

In [ ]:
# import os
# import json
# from PIL import Image, ImageDraw
# from surya.detection import DetectionPredictor

# # --- Input directory containing images ---
# IMAGE_DIR = "/content/drive/MyDrive/Thesis/surya/surya_input_images"

# # --- Initialize detection predictor once ---
# detection_predictor = DetectionPredictor(device="cuda")

# # --- Prepare output directory ---
# detected_dir = os.path.join(IMAGE_DIR, "detected_images")
# os.makedirs(detected_dir, exist_ok=True)

# # --- Loop over all image files in IMAGE_DIR ---
# for filename in os.listdir(IMAGE_DIR):
#     if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
#         continue  # skip non-image files

#     IMAGE_PATH = os.path.join(IMAGE_DIR, filename)
#     image = Image.open(IMAGE_PATH).convert("RGB")

#     # Run text line detection
#     detections = detection_predictor([image])[0]  # returns TextDetectionResult

#     # Prepare output file paths
#     img_name = os.path.splitext(filename)[0]
#     img_out_path = os.path.join(detected_dir, f"{img_name}_detected.jpg")
#     json_out_path = os.path.join(detected_dir, f"{img_name}_detections.json")

#     # Draw bounding boxes on image
#     draw = ImageDraw.Draw(image)
#     for polybox in detections.bboxes:
#         x_min, y_min, x_max, y_max = map(int, polybox.bbox)
#         draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
#         draw.text((x_min, max(0, y_min - 10)), f"{polybox.confidence:.2f}", fill="red")

#     # Save annotated image
#     image.save(img_out_path)

#     # Save detections as JSON
#     detections_list = []
#     for polybox in detections.bboxes:
#         detections_list.append({
#             "polygon": polybox.polygon,
#             "confidence": float(polybox.confidence),
#             "bbox": list(map(int, polybox.bbox))
#         })

#     with open(json_out_path, "w") as f:
#         json.dump(detections_list, f, indent=4)

#     print(f"✅ Processed: {filename}")
#     print(f"   Saved detected image → {img_out_path}")
#     print(f"   Saved detections JSON → {json_out_path}")


Detecting bboxes: 100%|██████████| 1/1 [00:00<00:00,  3.59it/s]


✅ Processed: original_image.jpg
   Saved detected image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/original_image_detected.jpg
   Saved detections JSON → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/original_image_detections.json


Detecting bboxes: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


✅ Processed: final_processed_image.png
   Saved detected image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/final_processed_image_detected.jpg
   Saved detections JSON → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/final_processed_image_detections.json


Detecting bboxes: 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]


...

✅ Processed: in1.jpg
   Saved detected image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1_detected.jpg
   Saved detections JSON → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1_detections.json


Detecting bboxes: 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


✅ Processed: product.jpg
   Saved detected image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/product_detected.jpg
   Saved detections JSON → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/product_detections.json


Detecting bboxes: 100%|██████████| 1/1 [00:00<00:00,  4.10it/s]

✅ Processed: today1.jpg
   Saved detected image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/today1_detected.jpg
   Saved detections JSON → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/today1_detections.json


In [ ]:
# import glob, shutil

# for ckpt in glob.glob("/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/today1/"):
#     shutil.rmtree(ckpt, ignore_errors=True)

CROP (works fast in GPU)

In [ ]:
import os
import json
from PIL import Image

# --- Paths for a single image ---
original_dir = os.path.dirname(IMAGE_PATH)
image_name = img_name = os.path.splitext(os.path.basename(IMAGE_PATH))[0]  # without extension
json_file = os.path.join(original_dir, "detected_images", f"{image_name}_detections.json")

# --- Open the corresponding original image ---
orig_img_path = os.path.join(original_dir, f"{image_name}.png")  # try .png
if not os.path.exists(orig_img_path):
    orig_img_path = os.path.join(original_dir, f"{image_name}.jpg")  # fallback .jpg
    if not os.path.exists(orig_img_path):
        raise FileNotFoundError(f"Original image not found for {image_name}")

image = Image.open(orig_img_path).convert("RGB")

# --- Load bounding boxes from JSON ---
with open(json_file, "r") as f:
    bboxes_data = json.load(f)

# --- Create output folder for cropped images ---
crop_dir = os.path.join(original_dir, "detected_images", image_name)
os.makedirs(crop_dir, exist_ok=True)

# --- Sort bounding boxes by y_min (top) ---
sorted_boxes = sorted(bboxes_data, key=lambda b: b["bbox"][1])

# --- Crop and save each bounding box ---
for i, box in enumerate(sorted_boxes, start=1):
    x_min, y_min, x_max, y_max = map(int, box["bbox"])
    cropped = image.crop((x_min, y_min, x_max, y_max))
    out_path = os.path.join(crop_dir, f"crop_{i}.jpg")
    cropped.save(out_path)

print(f"✅ Saved {len(sorted_boxes)} cropped images for {image_name} → {crop_dir}")


✅ Saved 16 cropped images for in1 → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1


for looping all images to crop

In [ ]:
# import os
# import json
# from PIL import Image

# # Directories
# original_dir = "/content/drive/MyDrive/Thesis/surya/surya_input_images"
# detected_dir = os.path.join(original_dir, "detected_images")

# # Loop over all detection JSON files
# for json_file in os.listdir(detected_dir):
#     if not json_file.endswith("_detections.json"):
#         continue

#     json_path = os.path.join(detected_dir, json_file)
#     img_name = json_file.replace("_detections.json", "")

#     # Open the corresponding original image (not the detected one)
#     orig_img_path = os.path.join(original_dir, f"{img_name}.png")  # adjust extension if needed
#     if not os.path.exists(orig_img_path):
#         orig_img_path = os.path.join(original_dir, f"{img_name}.jpg")
#         if not os.path.exists(orig_img_path):
#             print(f"⚠️ Original image not found for {img_name}, skipping")
#             continue

#     image = Image.open(orig_img_path).convert("RGB")

#     # Load bounding boxes from JSON
#     with open(json_path, "r") as f:
#         bboxes_data = json.load(f)

#     # Create output folder for cropped images
#     crop_dir = os.path.join(detected_dir, img_name)
#     os.makedirs(crop_dir, exist_ok=True)

#     # Sort bounding boxes by y_min (top)
#     sorted_boxes = sorted(bboxes_data, key=lambda b: b["bbox"][1])

#     # Crop and save each bounding box
#     for i, box in enumerate(sorted_boxes, start=1):
#         x_min, y_min, x_max, y_max = map(int, box["bbox"])
#         cropped = image.crop((x_min, y_min, x_max, y_max))
#         out_path = os.path.join(crop_dir, f"crop_{i}.jpg")
#         cropped.save(out_path)

#     print(f"✅ Saved {len(sorted_boxes)} cropped images for {img_name} → {crop_dir}")


✅ Saved 39 cropped images for product → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/product
✅ Saved 16 cropped images for in1 → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1
✅ Saved 12 cropped images for original_image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/original_image
✅ Saved 15 cropped images for today1 → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/today1
✅ Saved 53 cropped images for final_processed_image → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/final_processed_image
✅ Saved 22 cropped images for final_processed_image (1) → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/final_processed_image (1)


tr-ocr apply

In [ ]:
# !pip install transformers==4.44.2 accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForVision2Seq, AutoImageProcessor
from PIL import Image
import torch
import os

# =========================
# Load TrOCR model
# =========================
tokenizer = AutoTokenizer.from_pretrained("microsoft/trocr-large-printed")
model = AutoModelForVision2Seq.from_pretrained("microsoft/trocr-large-printed")
image_processor = AutoImageProcessor.from_pretrained("microsoft/trocr-large-printed")

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# =========================
# Folder containing cropped images for a single original image
# =========================
image_folder = os.path.join(original_dir, "detected_images")
original_image_name = "in1"  # the folder with cropped images
cropped_folder = os.path.join(image_folder, original_image_name)

# List all cropped images in that folder
image_paths = [os.path.join(cropped_folder, f)
               for f in os.listdir(cropped_folder)
               if f.lower().endswith((".png", ".jpg", ".jpeg"))]

# =========================
# Batch size
# =========================
batch_size = 1  # adjust based on GPU memory

# =========================
# Function to process a batch of images
# =========================
def ocr_batch(image_paths_batch):
    images = [Image.open(p).convert("RGB") for p in image_paths_batch]
    pixel_values = image_processor(images, return_tensors="pt").pixel_values.to(device)
    generated_ids = model.generate(pixel_values)
    texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return texts

# =========================
# Process all images in batches
# =========================
ocr_texts = []
for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    batch_texts = ocr_batch(batch_paths)
    ocr_texts.extend(batch_texts)

# =========================
# Print results
# =========================
for img_path, text in zip(image_paths, ocr_texts):
    print(f"{img_path}: {text}")


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_1.jpg: IG
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_2.jpg: INGREDIENTS:
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_3.jpg: POTATO, REFINED PALMOLEIN OIL, BENGAL GRAM
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_4.jpg: FLOUR (BESAN), POTATO FLAKES, POTATO STAR
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_5.jpg: TAPIOCA STARCH, TAPARY BEANS (MOTH DAL)
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_6.jpg: FLOUR, LODISED SALT, RED CHILLI POWDER,
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_7.jpg: BLACK SALT, ACIDITY REGULATOR (INS 330),
/content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/crop_8.jpg: BLACK PEPPER, CLOVE, CARDAMOM, DRIED
/content/drive/MyDrive/Thesis/surya/surya_inp

In [ ]:
import json

# =========================
# Print & Save results
# =========================
main_img_name = os.path.splitext(os.path.basename(cropped_folder))[0]
jsonl_out_path = os.path.join(cropped_folder, f"{main_img_name}.jsonl")

with open(jsonl_out_path, "w", encoding="utf-8") as f:
    for img_path, text in zip(image_paths, ocr_texts):
        img_name = os.path.basename(img_path)  # only file name, not full path
        record = {"image": img_name, "text": text}
        f.write(json.dumps(record, ensure_ascii=False) + "\n")  # write one JSON object per line
        print(f"{img_name}: {text}")

print(f"✅ OCR results saved to {jsonl_out_path}")


crop_1.jpg: IG
crop_2.jpg: INGREDIENTS:
crop_3.jpg: POTATO, REFINED PALMOLEIN OIL, BENGAL GRAM
crop_4.jpg: FLOUR (BESAN), POTATO FLAKES, POTATO STAR
crop_5.jpg: TAPIOCA STARCH, TAPARY BEANS (MOTH DAL)
crop_6.jpg: FLOUR, LODISED SALT, RED CHILLI POWDER,
crop_7.jpg: BLACK SALT, ACIDITY REGULATOR (INS 330),
crop_8.jpg: BLACK PEPPER, CLOVE, CARDAMOM, DRIED
crop_9.jpg: GINGER POWDER, DRIED GARLIC POWDER, REFINED
crop_10.jpg: SUGAR, CUMIN POWDER, BAY LEAVES,
crop_11.jpg: NUTMEG, CINNAMON AND MINT LEAVES
crop_12.jpg: CONTAINS ADDED NATURAL, NATURE IDENTICAL &
crop_13.jpg: ARTIFICIAL FLAVOURING SUBSTANCES (MINT)
crop_14.jpg: ALLERGY ADVICE:
crop_15.jpg: MAY CONTAIN PEANUT, TREE NUTS, SESAME
crop_16.jpg: SEED, MILK, SOYA, MUSTARD SEED & GLUTEN
✅ OCR results saved to /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/in1.jsonl


# extracting Ingredients

In [ ]:
# copy from drive to colab memory
!cp -r "/content/drive/MyDrive/Thesis/ingredient-model" "./ingredient-model"

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_path = "./ingredient-model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

ner_pipeline = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0  # GPU
)


Device set to use cuda:0


In [ ]:
import json

input_file = jsonl_out_path

texts = []
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        texts.append(data["text"])
# Merge with newline and convert to lowercase
merged_text = " ".join(texts).lower()
print(merged_text)

ig ingredients: potato, refined palmolein oil, bengal gram flour (besan), potato flakes, potato star tapioca starch, tapary beans (moth dal) flour, lodised salt, red chilli powder, black salt, acidity regulator (ins 330), black pepper, clove, cardamom, dried ginger powder, dried garlic powder, refined sugar, cumin powder, bay leaves, nutmeg, cinnamon and mint leaves contains added natural, nature identical & artificial flavouring substances (mint) allergy advice: may contain peanut, tree nuts, sesame seed, milk, soya, mustard seed & gluten


In [ ]:
results = ner_pipeline(merged_text, batch_size=16)
for r in results:
    print(r)

{'entity_group': 'ING', 'score': np.float32(0.9998621), 'word': 'potato , refined palmolein oil , bengal gram flour ( besan ) , potato flakes , potato star tapioca starch , tapary beans ( moth dal ) flour , lodised salt , red chilli powder , black salt , acidity regulator ( ins 330 ) , black pepper , clove , cardamom , dried ginger powder , dried garlic powder , refined sugar , cumin powder , bay leaves , nutmeg , cinnamon and mint leaves', 'start': 16, 'end': 368}


In [ ]:
import json
import os

# Example image name
image_name = main_img_name   # <-- set dynamically if you loop over images

# Directory in Google Drive where files will be stored
output_dir = cropped_folder
os.makedirs(output_dir, exist_ok=True)

# === Save raw predictions JSONL ===
pred_file = os.path.join(output_dir, f"{image_name}_ingredients_pred.jsonl")
with open(pred_file, "w", encoding="utf-8") as f:
    for r in results:
        # Convert numpy.float32 → normal float
        r["score"] = float(r["score"])
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"Saved raw predictions → {pred_file}")


# === Filter texts by threshold ===
def filter_texts(jsonl_path, threshold=0.95):
    filtered = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            if data["score"] >= threshold:
                filtered.append(data["word"])
    return filtered


threshold = 0.97  # <-- user-given threshold
filtered_texts = filter_texts(pred_file, threshold=threshold)

# === Save filtered texts to TXT file ===
final_jsonl_file = os.path.join(output_dir, f"{image_name}_final_ingredients.jsonl")
with open(final_jsonl_file, "w", encoding="utf-8") as f:
    for txt in filtered_texts:
        json_line = {"ingredient": txt}
        f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

print(f"Saved filtered ingredients → {final_jsonl_file}")


Saved raw predictions → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/in1_ingredients_pred.jsonl
Saved filtered ingredients → /content/drive/MyDrive/Thesis/surya/surya_input_images/detected_images/in1/in1_final_ingredients.jsonl


detects but the recognision takes lifetime

In [ ]:
# from PIL import Image
# from surya.foundation import FoundationPredictor
# from surya.recognition import RecognitionPredictor
# from surya.detection import DetectionPredictor
# IMAGE_PATH = "/content/binary_text.png"
# image = Image.open(IMAGE_PATH)
# foundation_predictor = FoundationPredictor()
# recognition_predictor = RecognitionPredictor(foundation_predictor)
# detection_predictor = DetectionPredictor()

# predictions = recognition_predictor([image], det_predictor=detection_predictor)

In [ ]:
# import glob, shutil

# for ckpt in glob.glob("./results"):
#     shutil.rmtree(ckpt, ignore_errors=True)

# LANGUAGE Detection (accuracy unacceptable)

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.0-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.0-py3-none-any.whl (292 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4508431 sha256=6fda5d03ba9426ed92428ffda2ff9e06629c2c36d84f9bb7590887225977d398
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2025-08-17 21:31:33--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.15, 108.157.254.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin’

lid.176.bin         100%[===================>] 125.18M   196MB/s    in 0.6s    

2025-08-17 21:31:33 (196 MB/s) - ‘lid.176.bin’ saved [131266198/131266198]



In [ ]:
import fasttext
import json
import os

# =========================
# Load fastText language identification model
# =========================
lang_model_path = "/content/drive/MyDrive/Thesis/surya/fasttext/lid.176.bin"
lang_model = fasttext.load_model(lang_model_path)

# =========================
# OCR outputs from TrOCR
# =========================
# ocr_texts = ["some text from crop_1.jpg", "another text from crop_2.jpg", ...]

# =========================
# Predict languages
# =========================
pred_labels, pred_probs = lang_model.predict(ocr_texts)  # batch prediction

# =========================
# Combine results
# =========================
results = []
for text, label, prob in zip(ocr_texts, pred_labels, pred_probs):
    lang = label[0].replace("__label__", "")  # remove fastText prefix
    lang_conf = float(prob[0])
    results.append({
        "text": text,
        "lang": lang,
        "lang_conf": lang_conf
    })

# =========================
# Print results
# =========================
for r in results:
    print(r)

# =========================
# Optional: Save to JSONL
# =========================
# jsonl_path = os.path.join(os.path.dirname(lang_model_path), "ocr_lang_results.jsonl")
# with open(jsonl_path, "w", encoding="utf-8") as f:
#     for r in results:
#         f.write(json.dumps(r, ensure_ascii=False) + "\n")

# print(f"✅ Language detection results saved to {jsonl_path}")


{'text': 'IG', 'lang': 'pt', 'lang_conf': 0.4512484073638916}
{'text': 'INGREDIENTS:', 'lang': 'en', 'lang_conf': 0.33771204948425293}
{'text': 'POTATO, REFINED PALMOLEIN OIL, BENGAL GR', 'lang': 'ca', 'lang_conf': 0.18098032474517822}
{'text': 'FLOUR (BESAN), POTATO FLAKES, POTATO', 'lang': 'en', 'lang_conf': 0.778530478477478}
{'text': 'TAPIOCA STARCH, TAPARY BEANS (MOTH DAL)', 'lang': 'en', 'lang_conf': 0.556109607219696}
{'text': 'FLOUR, LODISED SALT, RED CHILLI POWDER,', 'lang': 'en', 'lang_conf': 0.5090453624725342}
{'text': 'BLACK SALT, ACIDITY REGULATOR (INS 330),', 'lang': 'pl', 'lang_conf': 0.17589902877807617}
{'text': 'BLACK PEPPER, CLOVE, CARDAMOM, DRIED', 'lang': 'fr', 'lang_conf': 0.1322389543056488}
{'text': 'GINGER POWDER, DRIED GARLIC POWDER, REFINED', 'lang': 'en', 'lang_conf': 0.38636747002601624}
{'text': 'SUGAR, CUMIN POWDER, BAY LEAVES,', 'lang': 'en', 'lang_conf': 0.41062816977500916}
{'text': 'NUTMEG, CINNAMON AND MINT LEAVES', 'lang': 'en', 'lang_conf': 0.9107

# *DETECTS* PARA. WORKS FINE

In [ ]:
# !surya_layout /content/binary_text.png --images

donot work

In [ ]:
# from PIL import Image, ImageDraw
# from surya.layout import LayoutPredictor

# IMAGE_PATH = "/content/binary_text.png"
# image = Image.open(IMAGE_PATH)

# # Initialize layout predictor
# layout_predictor = LayoutPredictor()

# # Run layout prediction (returns dict for each image)
# layout_predictions = layout_predictor([image])[0]

# # Copy image for drawing
# drawn = image.copy()
# draw = ImageDraw.Draw(drawn)

# # Loop over bounding boxes and labels
# for bbox, label in zip(layout_predictions["bboxes"], layout_predictions["labels"]):
#     x_min, y_min, x_max, y_max = bbox
#     # Draw rectangle
#     draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)
#     # Draw label text above the box
#     draw.text((x_min, max(0, y_min-12)), label, fill="red")

# # Save & show
# drawn.save("layout_detected.png")
# drawn.show()


Recognizing layout: 100%|██████████| 1/1 [00:20<00:00, 20.02s/it]


TypeError: 'LayoutResult' object is not subscriptable

takes lifetime

In [ ]:
!surya_ocr /content/binary_text.png --images

2025-08-17 10:38:46.209770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755427126.251209    9533 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755427126.264737    9533 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755427126.299677    9533 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755427126.299740    9533 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755427126.299745    9533 computation_placer.cc:177] computation placer alr

In [ ]:
!pip install contextualSpellCheck

In [ ]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)
doc = nlp(filtered_texts)

print(doc._.performed_spellCheck) #Should be True
print(doc._.outcome_spellCheck) #Income was $9.4 million compared to the prior year of $2.7 million.